In [ ]:
import os
import glob
import pandas as pd
import tensorflow as tf
import xml.etree.ElementTree as ET
from PIL import Image

In [ ]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('xml_dir', 'Annotations/xmls',
                           'Training data directory')

In [ ]:
data = pd.read_csv('TownCentre-groundtruth.top', header=None)
data

In [ ]:
def get_xml_files(xml_dir):
    """"Reads addresses and labels from `data_dir`.
    The `tf.gfile.Glob` module return a list of paths matching a pathname pattern
    
    Args:
        data_dir = Path to the `input image folder`.
    
    Returns:
        sorted_matching_files: list, address of each picture in the train folder.
    """
    cur_dir = os.getcwd()
    new_dir = os.chdir(xml_dir)
    
    matching_files = glob.glob('*.xml')
    sorted_matching_files = sorted(matching_files)
    
    # Go back to the original directory
    os.chdir(cur_dir)
    
    return sorted_matching_files

In [ ]:
def check_annotation(xml_dir, filename):
    # Create a `ElementTree` class to extract information
    xml_path = os.path.join(xml_dir, filename)
    root = ET.parse(xml_path).getroot()
    
    # Get width and height in the annotation
    size_attributes = root.find('size')
    img_width = int(size_attributes.find('width').text)
    img_height = int(size_attributes.find('height').text)

    # Check bounding boxes
    for obj in root.findall('object'):
        ele = obj.find('bndbox')
        if ele is None:
            continue

        xmin = int(ele.find('xmin').text)
        ymin = int(ele.find('ymin').text)
        xmax = int(ele.find('xmax').text)
        ymax = int(ele.find('ymax').text)
        
        if xmin > img_width:
            print(f'Dimension mismatch: {xml_path}')
            print(f'xmin: {xmin}')
        if xmax > img_width:
            print(f'Dimension mismatch: {xml_path}')
            print(f'xmax: {xmax}')
        if ymin > img_height:
            print(f'Dimension mismatch: {xml_path}')
            print(f'ymin: {ymin}')
        if ymax > img_height:
            print(f'Dimension mismatch: {xml_path}')
            print(f'ymax: {ymax}')

In [ ]:
def main(unused_argv):
    filenames = get_xml_files(FLAGS.xml_dir)
    for filename in filenames:
        check_annotation(FLAGS.xml_dir, filename)


if __name__ == '__main__':
    tf.app.run()